## Overview of Assignment 4

This assignment focuses on exploring and implementing advanced concepts and techniques in information retrieval. The primary objectives are to build Retrieval Augumentation Generation, and learn about Language Models

## Enter your details below

## Name

## Banner ID

## GitHub Link of your Assingment 4

## Q1 : Setting up the libraries and the environment

In [ ]:
!pip install -q langchain
!pip install -q faiss-cpu     
!pip install -q sentence-transformers
!pip install -q transformers
!pip install -q datasets
!pip install -q tiktoken
!pip install -q openai
!pip install -q chromadb     


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\AVuser\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset

import os
import numpy as np
import torch

## Q2:  Data Preprocessing and Model Selection

## Q3: Implementing RAG using LangChain for different queries

## Q4 : Modify and evaluate the different components of RAG

## Q5: Selecting and implementing a pretrained model for a new task